In [1]:
# start docker deamon in terminal
# !sudo docker login
# OR - becuase the jupyther with allow root
# !docker login

In [1]:
!pwd

/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna


In [2]:
!ls bin

rna_qc_plot.py	scrna_plot_qc_metrics.R  test.py


In [48]:
!chmod +x ./bin/scrna_qc_metrics.py

In [4]:
!gcloud auth list
# gcloud config set account


                  Credentialed Accounts
ACTIVE  ACCOUNT
        886987709752-compute@developer.gserviceaccount.com
*       eila-batch@hai-gcp-genomic.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [7]:
import os
# os.environ["TOWER_ACCESS_TOKEN"] = 'XXX'
# os.environ["TOWER_ACCESS_TOKEN"] = 'eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz'

!echo $TOWER_ACCESS_TOKEN

eyJ0aWQiOiA4Mjk3fS5jNDNmYzA4NTg2MjAxNDQzOTAwYmUzNWEzZTY0MGMyZTc0OWE3M2Mz


In [8]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [9]:
import os
os.getcwd()

'/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna'

In [10]:
%%writefile nextflow_local.config

includeConfig 'conf/conda.config'


manifest {
  description = 'A nexflow pipeline for fg single cell' 
  mainScript = 'nf_wf_fg_singlecell_rna.nf'
}

profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  conda_docker_profile {
    includeConfig 'conf/conda_docker.config'
    executor = 'local'
    }

  docker_local {
    includeConfig 'conf/docker.config'
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
  }
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work/RNA'
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = false
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }
}

// Define ENV_SYNAPSE_TOKEN as a parameter
params.ORGANISM = 'human'
params.FASTQS_SPEC_CH = '/Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/rna_fastqs_files_local.csv'
params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
params.ENV_SYNAPSE_TOKEN = "nextflow secrets get SYNAPSE".execute().text.trim()
params.SYNAPSE_CONFIG = '/Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/synapse.config'
params.SYNAPSE_IDS = '/Users/eilaarich-landkof-stanford/Documents/Code/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/synapse_ids.csv'
params.SC_RNA_QC_SCRIPT='scrna_qc_metrics.py'
params.SUBPOOL = 'no_subpool'
params.UMI_CUTOFF=2
params.GENE_CUTOFF=3
params.ANNDATA_FILE = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna/adata.h5ad'


Overwriting nextflow_local.config


In [11]:
!ls -lt /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna/adata.h5ad

-rw-r--r-- 1 eila eila 391817928 Nov 13 23:23 /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna/adata.h5ad


In [12]:
!nextflow info

  Version: 23.10.0 build 5889
  Created: 15-10-2023 15:07 UTC 
  System: Linux 5.10.0-26-cloud-amd64
  Runtime: Groovy 3.0.19 on OpenJDK 64-Bit Server VM 11.0.13+7-b1751.21
  Encoding: UTF-8 (UTF-8)



In [13]:
# !ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell

In [14]:
%%writefile nextflow_vm.config

includeConfig 'conf/conda.config'


manifest {
  description = 'A nexflow pipeline for fg single cell' 
  mainScript = 'nf_wf_fg_singlecell_rna.nf'
}

profiles {
  conda_profile {
    includeConfig 'conf/conda.config'
    conda.enabled = true
    executor = 'local'
    }

  docker_local {
    includeConfig 'conf/docker.config'
    docker.enabled = true
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    executor = 'local'
  }
  google_batch_profile {
    workDir = 'gs://eila-nextflow-bucket/work/RNA'
    includeConfig 'conf/docker-google-batch.config'
    docker.enabled = true
    wave.enabled = true
    fusion.enabled = true
    process.scratch = false
    executor = 'google-batch'
    docker.envWhitelist=['ENV_SYNAPSE_TOKEN','TOWER_ACCESS_TOKEN']
    google.project = 'hai-gcp-genomic'
    google.location = 'us-central1'
    google.batch.spot = true
    google.batch.serviceAccountEmail='eila-batch@hai-gcp-genomic.iam.gserviceaccount.com'
  }
    
}

// Define ENV_SYNAPSE_TOKEN as a parameter
params.ORGANISM = 'human'
params.FASTQS_SPEC_CH = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/rna_fastqs_files_vm-dogma-dig.csv'
params.GENOME_FASTA='http://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz'
params.GENOME_GZ_GTF='http://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz'
params.ENV_SYNAPSE_TOKEN = "nextflow secrets get SYNAPSE".execute().text.trim()
params.SYNAPSE_CONFIG = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/synapse.config'
params.SYNAPSE_IDS = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/synapse_ids.csv'
params.SUBPOOL = 'no_subpool'
params.UMI_CUTOFF=2
params.GENE_CUTOFF=3
params.SC_RNA_QC_SCRIPT='test.py'
params.ANNDATA_FILE = '/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna/adata.h5ad'


Overwriting nextflow_vm.config


In [15]:
# !cat /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/rna_fastqs_files_vm-dogma-dig.csv

In [16]:
# !head /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/nf_spec_dogmaseq_dig.yaml

In [17]:
# !head /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/specs/dogmaseq-dig/RNA-737K-arc-v1.txt

In [18]:
# !ls /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna


In [19]:
# !rm -r /home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_workflow/workflow_fg_single_cell_rna/work


# !gsutil -m rm -r gs://eila-nextflow-bucket/work/RNA




In [45]:

!nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume 8af1ba30-6606-4dfd-a851-c6d0b74bdd43


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local -resume



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile  -with-tower

# docker_local



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile




# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm_anvil.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile google_batch_profile
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile docker_profile

N E X T F L O W  ~  version 23.10.0
Launching `nf_wf_fg_singlecell_rna.nf` [desperate_banach] DSL2 - revision: 247da0ef81
Nov 14, 2023 6:11:50 AM com.google.auth.oauth2.DefaultCredentialsProvider warnAboutProblematicCredentials
null
/home/eila/git_igvf/cellatlas-scrna-seq-pipeline/nextflow/nf_data/nf_wf_fg_singlecell/rna_fastqs_files_vm-dogma-dig.csv
[-        ] process > run_seqspec_modify_rna     -
[-        ] process > run_seqspec_print          -
[-        ] process > run_seqspec_index_rna_kb   -
[-        ] process > run_kb_ref                 -
[-        ] process > run_kb_count               -
[-        ] process > scrna_calculate_qc_metrics -
after run_seqspec_modify_rna
after run_seqspec_print
after run_seqspec_index_rna_kb
technology is $technology
after genome_fasta_ch
after genome_gtf_ch
after run_kb_ref
after get organism
after run_kb_count
scrna_qc_metrics_tsv is: DataflowBroadcast around DataflowStream[?]

Monitor the execution with Nextflow Tower using this URL: https:/

In [46]:
!cat .nextflow.log

Nov-14 06:11:45.547 [main] DEBUG nextflow.cli.Launcher - $> nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume 8af1ba30-6606-4dfd-a851-c6d0b74bdd43
Nov-14 06:11:45.898 [main] INFO  nextflow.cli.CmdRun - N E X T F L O W  ~  version 23.10.0
Nov-14 06:11:45.941 [main] DEBUG nextflow.plugin.PluginsFacade - Setting up plugin manager > mode=prod; embedded=false; plugins-dir=/home/eila/.nextflow/plugins; core-plugins: nf-amazon@2.1.4,nf-azure@1.3.2,nf-cloudcache@0.3.0,nf-codecommit@0.1.5,nf-console@1.0.6,nf-ga4gh@1.1.0,nf-google@1.8.3,nf-tower@1.6.3,nf-wave@1.0.0
Nov-14 06:11:45.960 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Enabled plugins: []
Nov-14 06:11:45.962 [main] INFO  o.pf4j.DefaultPluginStatusProvider - Disabled plugins: []
Nov-14 06:11:45.969 [main] INFO  org.pf4j.DefaultPluginManager - PF4J version 3.4.1 in 'deployment' mode
Nov-14 06:11:45.989 [main] INFO  org.pf4j.AbstractPluginManager - No plugins
Nov-14 06:11:

In [47]:
import datetime
now = datetime.datetime.now()
print(now)

2023-11-14 06:12:16.482265


In [29]:

# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -with-tower -resume 
# 8af1ba30-6606-4dfd-a851-c6d0b74bdd43


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume 886dc4ef-ff60-46d0-aecb-8f1ab56296a3

# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume


# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume



# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile docker_local -resume





# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume




# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_local.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow_vm.config -profile google_batch_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile docker_profile -resume
# !nextflow run nf_wf_fg_singlecell_rna.nf -c nextflow.config -profile google_batch_profile -resume

In [30]:
# !nextflow log
# use the session id with the resume

In [31]:
# !cat .nextflow.log 

In [529]:
import datetime
now = datetime.datetime.now()
print(now)

2023-11-14 01:10:18.537013
